**Exercise 1.5.** In Example 1.2.4, we considered an agent who sold the lookback option for $V_0 = 1.376$ and bought $\Delta_0 = 0.1733$ shares of stock at time zero. At time one, if the stock goes up, she has a portfolio valued at $V_1(H) = 2.24$. Assume that she now takes a position of $\Delta_1(H) = \frac{V_2(HH) − V_2(HT)}{S_2(HH) − S_2(HT)}$ in the stock. Show that, at time two, if the stock goes up again, she will have a portfolio valued at $V_2(HH) = 3.20$, whereas if the stock goes down, her portfolio will be worth $V_2(HT) = 2.40$. Finally, under the assumption that the stock goes up in the first period and down in the second period, assume the agent takes a position of $\Delta_2(HT) = \frac{V_3(HTH) − V_3(HTT)}{S_3(HTH)−S_3(HTT)}$ in the stock. Show that, at time three, if the stock goes up in the third period, she will have a portfolio valued at $V_3(HTH) = 0$, whereas if the stock goes down, her portfolio will be worth $V_3(HTT) = 6$. In other words, she has hedged her short position in the option.

---

The excercise asks us to prove:

$$
\begin{cases}
X_2(HH) = V_2(HH) = 3.20 \\
X_2(HT) = V_2(HT) = 2.40 \\
X_3(HTH) = V_3(HTH) = 0 \\
X_3(HTT) = V_3(HTT) = 6
\end{cases}.
$$

---

As given by Example 1.2.4 and the excercise, we define $S_0$, $r$, $u$, $d$ and $V_0$:

In [1]:
S₀ = 4
r = 1/4
u = 2
d = 1/2
p̃ = q̃ = 1/2
V₀ = 1.376;

We define the Toss enum:

In [2]:
@enum Toss H T

Given $S_0$, $u$ and $d$ we define $S(\omega_1 \omega_2 \ldots \omega_i) = S_i$:

In [3]:
function S(ω::Toss...)
    i = length(ω)
    if i == 0
        return S₀
    else
        Sᵢ₋₁ = S(ω[1 : end - 1]...)
        if ω[i] == H
            return u*Sᵢ₋₁
        else
            return d*Sᵢ₋₁
        end
    end
end;

Given the lookback option payoff

$$V_3 = \max_{0 \leq n \leq 3} S_n - S_3$$

and the formula (1.2.16)

$$V_i = \frac{1}{1 + r} (\tilde p V_{i + 1}(H) + \tilde q V_{i + 1}(T)),$$

we define $V(\omega_1 \omega_2 \ldots \omega_i) = V_i$:

In [4]:
function V(ω::Toss...)
    i = length(ω)
    if i == 3
        S₁ = S(ω[1])
        S₂ = S(ω[1 : 2]...)
        S₃ = S(ω[1 : 3]...)
        # lookback option payoff
        return max(S₀ - S₃, S₁ - S₃, S₂ - S₃, S₃ - S₃)
    elseif i < 3
        # formula (1.2.16)
        return 1/(1 + r)*(p̃*V(ω..., H) + q̃*V(ω..., T))
    else
        error("out of domain parameter")
    end
end;

From the formula (1.2.17)

$$\Delta_i = \frac{V_{i + 1}(H) - V_{i + 1}(T)}{S_{i + 1}(H) - S_{i + 1}(T)},$$

we define $\Delta(\omega_1 \omega_2 \ldots \omega_i) = \Delta_i$:

In [5]:
Δ(ω::Toss...) = (V(ω..., H) - V(ω..., T))/(S(ω..., H) - S(ω..., T));

The initial wealth is obtained by selling the lookback option for $V_0$:

$$X_0 = V_0.$$

Given the *wealth equation* formula (1.2.14)

$$X_{n + 1} = \Delta_n S_{n + 1} + (1 + r)(X_n - \Delta_n S_n).$$

Let

$$i = n + 1.$$

We can rewrite the *wealth equation* as

$$X_i = \Delta_{i - 1} S_i + (1 + r)(X_{i - 1} - \Delta_{i - 1} S_{i - i}).$$

So, we define $X(\omega_1 \omega_2 \ldots \omega_i) = X_i$:

In [6]:
function X(ω::Toss...)
    i = length(ω)
    if i == 0
        # X₀ = V₀
        return V₀
    else
        Δᵢ₋₁ = Δ(ω[1 : end - 1]...)
        Sᵢ = S(ω...)
        Xᵢ₋₁ = X(ω[1 : end - 1]...)
        Sᵢ₋₁ = S(ω[1 : end - 1]...)
        # formula (1.2.14)
        return Δᵢ₋₁*Sᵢ + (1 + r)*(Xᵢ₋₁ - Δᵢ₋₁*Sᵢ₋₁)
    end
end;

Since we are dealing with floating point numbers we check equalities with `isapprox` function using the equivalent `≈` operator.

$X_2(HH) = V_2(HH) = 3.20$:

In [7]:
X(H, H)

3.1999999999999997

In [8]:
X(H, H) ≈ V(H, H)

true

$X_2(HT) = V_2(HT) = 2.40$:

In [9]:
X(H, T)

2.4

In [10]:
X(H, T) ≈ V(H, T)

true

$X_3(HTH) = V_3(HTH) = 0$:

In [11]:
X(H, T, H)

0.0

In [12]:
X(H, T, H) ≈ V(H, T, H)

true

$X_3(HTT) = V_3(HTT) = 6$:

In [13]:
X(H, T, T)

6.0

In [14]:
X(H, T, T) ≈ V(H, T, T)

true

$$\square$$